In [2]:
import sys
sys.path.append('On_Multiple_Semantics_for_Declarative_Database_Repairs/')
import On_Multiple_Semantics_for_Declarative_Database_Repairs

from database_generator.dba import DatabaseEngine
from Semantics.end_sem import EndSemantics
from Semantics.stage_sem import StageSemantics
from Semantics.step_sem import StepSemantics
from Semantics.independent_sem import IndependentSemantics

# specify the schema
mas_schema = {"author": ('aid',
                         'name',
                         'oid'),
              "publication": ('pid',
                              'title',
                              'year'),

              "writes": ('aid', 'pid'),

              "organization": ('oid',
                               'name'),

              "conference": ('cid',
                             'mas_id',
                             'name',
                             'full_name',
                             'homepage',
                             'paper_count',
                             'citation_count',
                             'importance'),

              "domain_conference" : ('cid', 'did'),

              "domain" : ('did',
                          'name',
                          'paper_count',
                          'importance'),

              "cite" : ('citing', 'cited')
              }


def read_rules(rule_file):
    """read programs from txt file"""
    all_programs = []
    with open(rule_file) as f:
        rules = []
        for line in f:
            if line.strip():
                tbl, r = line.split("|")
                rules.append((tbl, r[:-2]))
            else:
                all_programs.append([r for r in rules])
                rules = []
        all_programs.append(rules)
    return all_programs

# load delta programs
programs = read_rules("On_Multiple_Semantics_for_Declarative_Database_Repairs/data/mas/join_programs.txt")

# start the database
db = DatabaseEngine("cr")

tbl_names = ["organization", "author", "publication", "writes", "cite"]
subfolder = "mas/"

def database_reset(db):
    """reset the database"""
    res = db.execute_query("select current_database();")
    db_name = res[0][0]
    db.delete_tables(tbl_names)
    # db.close_connection()
    db = DatabaseEngine(db_name)
    db.load_database_tables(tbl_names)

# choose a delta program from the programs file
rules = programs[0]
print("delta program:", rules)

# reset the database between runs
database_reset(db)

end_sem = EndSemantics(db, rules, tbl_names)
end_semantics_result = end_sem.find_mss()
print("result for end semantics:", end_semantics_result)

# reset the database between runs
database_reset(db)

stage_sem = StageSemantics(db, rules, tbl_names)
stage_semantics_result = stage_sem.find_mss()
print("result for stage semantics:", stage_semantics_result)

# reset the database between runs
database_reset(db)

step_sem = StepSemantics(db, rules, tbl_names)
step_semantics_result = step_sem.find_mss(mas_schema)
print("result for step semantics:", step_semantics_result)

# reset the database between runs
database_reset(db)

ind_sem = IndependentSemantics(db, rules, tbl_names)
ind_semantics_result = ind_sem.find_mss(mas_schema)
print("result for independent semantics:", ind_semantics_result)


delta program: [('cite', 'SELECT cite.* FROM cite WHERE cite.citing > 10000 AND cite.citing < 20000')]
result for end semantics: {('cite', (19008, 469621)), ('cite', (18229, 2731795)), ('cite', (18064, 2731585)), ('cite', (18414, 1896635)), ('cite', (18674, 1619790)), ('cite', (13369, 2465293)), ('cite', (18674, 1436674)), ('cite', (18050, 2738634)), ('cite', (18711, 2571942)), ('cite', (12027, 469547)), ('cite', (18647, 1906022)), ('cite', (18480, 1896572)), ('cite', (14642, 1231685)), ('cite', (18042, 1771348)), ('cite', (18642, 1913772)), ('cite', (18049, 1922757)), ('cite', (13840, 2369732)), ('cite', (18106, 1922757)), ('cite', (13438, 2429322)), ('cite', (13719, 919010)), ('cite', (19520, 1921758)), ('cite', (18628, 2317634)), ('cite', (13273, 1507178)), ('cite', (19210, 1012360)), ('cite', (18575, 2682640)), ('cite', (10136, 264659)), ('cite', (18566, 1620651)), ('cite', (11036, 2340568)), ('cite', (10181, 1145228)), ('cite', (14106, 1485661)), ('cite', (15442, 2769223)), ('cite

result for step semantics: {('cite', (19008, 469621)), ('cite', (18229, 2731795)), ('cite', (18414, 1896635)), ('cite', (18674, 1619790)), ('cite', (18064, 2731585)), ('cite', (13369, 2465293)), ('cite', (18674, 1436674)), ('cite', (18480, 1896572)), ('cite', (18050, 2738634)), ('cite', (18647, 1906022)), ('cite', (12027, 469547)), ('cite', (18711, 2571942)), ('cite', (14642, 1231685)), ('cite', (18642, 1913772)), ('cite', (18042, 1771348)), ('cite', (18049, 1922757)), ('cite', (13840, 2369732)), ('cite', (18106, 1922757)), ('cite', (13438, 2429322)), ('cite', (13719, 919010)), ('cite', (18628, 2317634)), ('cite', (19520, 1921758)), ('cite', (13273, 1507178)), ('cite', (19210, 1012360)), ('cite', (18575, 2682640)), ('cite', (10136, 264659)), ('cite', (18566, 1620651)), ('cite', (11036, 2340568)), ('cite', (14106, 1485661)), ('cite', (10181, 1145228)), ('cite', (15442, 2769223)), ('cite', (19268, 1881764)), ('cite', (18635, 1461527)), ('cite', (18406, 1459129)), ('cite', (18434, 2240967

In [3]:
%%capture
pip install psycopg2